In [ ]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

In [ ]:
##
# Configure the model training policy:
#
model_development.configure_training_policy()

In [ ]:
##
# Load and preprocess data:
#
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv', True)
print([cell_images.shape, len(cell_labels), len(cell_types)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

[(2624, 300, 300), 2624, 2624]


interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

In [ ]:
##
# Prepare data for model training:
#
train_test_split = int(0.85 * cell_images.shape[0])
X_train, Y_train, Z_train  = data_handling.preprocess_data(\
    cell_images[: train_test_split], 
    cell_labels[: train_test_split], 
    cell_types[: train_test_split])
X_test, Y_test, Z_test = \
    cell_images[train_test_split : ], \
    cell_labels[train_test_split : ], cell_types[train_test_split : ]

print([X_train.shape, len(Y_train), len(Z_train)])
print([X_test.shape, len(Y_test), len(Z_test)])

[(2230, 300, 300, 3), 2230, 2230]
[(394, 300, 300), 394, 394]


In [ ]:
##
# Train classification model (first phase):
#
optimizer = model_development.create_optimizer('nadam')
nasnetlarge = \
    model_development.nasnetlarge(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = False)

history, training_time = model_development.train_classification_model(\
    training_phase = 1, model = nasnetlarge, 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'NasNetLarge', version = '1.00.00_fs', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 1.0448 - accuracy: 0.6396 - precision: 0.6381 - recall: 0.4902
Epoch 1: val_accuracy improved from -inf to 0.47164, saving model to ../models/weights\InceptionResNetv2_20221003_fs.hdf5
119/119 [==============================] - 42s 226ms/step - loss: 1.0448 - accuracy: 0.6396 - precision: 0.6381 - recall: 0.4902 - val_loss: 1.2428 - val_accuracy: 0.4716 - val_precision: 0.4701 - val_recall: 1.0000
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5662 - accuracy: 0.7061 - precision: 0.7627 - recall: 0.5190
Epoch 2: val_accuracy improved from 0.47164 to 0.73134, saving model to ../models/weights\InceptionResNetv2_20221003_fs.hdf5
119/119 [==============================] - 23s 194ms/step - loss: 0.5662 - accuracy: 0.7061 - precision: 0.7627 - recall: 0.5190 - val_loss: 0.5582 - val_accuracy: 0.7313 - val_precision: 0.7913 - val_recall: 0.5796
Epoch 3/200
119/119 [==============================] 

In [ ]:
##
#Train classification model (second phase):
#
optimizer = model_development.create_optimizer('adam')
history, training_time = \
    model_development.train_classification_model(training_phase = 2, 
    model = '../models/weights/NasNetLarge_1.00.00_fs.hdf5', 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'NasNetLarge', version = '1.00.00_ss', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 19.8557 - accuracy: 0.5673 - precision: 0.5158 - recall: 0.8858
Epoch 1: val_accuracy improved from -inf to 0.54328, saving model to ../models/weights\InceptionResNetv2_20221003_ss.hdf5
119/119 [==============================] - 70s 456ms/step - loss: 19.8557 - accuracy: 0.5673 - precision: 0.5158 - recall: 0.8858 - val_loss: 7.3791 - val_accuracy: 0.5433 - val_precision: 0.5069 - val_recall: 0.9299
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 3.3450 - accuracy: 0.6818 - precision: 0.6361 - recall: 0.7116
Epoch 2: val_accuracy did not improve from 0.54328
119/119 [==============================] - 49s 413ms/step - loss: 3.3450 - accuracy: 0.6818 - precision: 0.6361 - recall: 0.7116 - val_loss: 5.6563 - val_accuracy: 0.5104 - val_precision: 0.4882 - val_recall: 0.9236
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 1.5309 - accuracy: 0.6654 - precision: 0.6395 - recall